In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [11]:
#Dados de Entrada
agents = ['Cons 1', 'Cons 2', 'Prod A', 'Prod B']
Na = len(agents)
p_max = [100, 50, 100, 80]
p_min = [0, 0, 0, 0]
agent_sign = [1, 1, -1, -1] #1: Consumer; -1: Producer
agent_conexion = pd.DataFrame([[0, 0, 1, 1],
                               [0, 0, 1, 1],
                               [1, 1, 0, 0],
                               [1, 1, 0, 0]], columns=agents, index=agents)

agent_conexion_dict = {(agents[i], agents[j]): agent_conexion.loc[agents[i], agents[j]] for i in range(Na) for j in range(Na)}
Ng = sum(1 for p in agent_sign if p == -1)
Nc = sum(1 for p in agent_sign if p == 1)
c_ener = [40, 35, 12, 20]

In [12]:
model = ConcreteModel()

#Sets:
model.agent = Set(initialize = agents)

#Parâmetros:
model.p_max = Param(model.agent, initialize = {agents[i]: p_max[i] for i in range(Na)})
model.p_min = Param(model.agent, initialize = {agents[i]: p_min[i] for i in range(Na)})
model.agent_sign = Param(model.agent, initialize = {agents[i]: agent_sign[i] for i in range(Na)})
model.c_ener = Param(model.agent, initialize = {agents[i]: c_ener[i] for i in range(Na)})
model.agent_conexion = Param(model.agent, model.agent, initialize = agent_conexion_dict)

In [13]:
#Variáveis:
model.p = Var(model.agent, model.agent, domain = NonNegativeReals)
model.P = Var(model.agent, domain = NonNegativeReals)

In [14]:
#Restrições:

def balanco_agente(model, agente):
    return sum(model.p[agente, i] for i in model.agent) == model.P[agente]
model.balanco_agente = Constraint(model.agent, rule = balanco_agente)

def limites_tecnicos_rule(model, agente):
    return (model.p_min[agente], model.P[agente], model.p_max[agente])
model.limites_tecnicos = Constraint(model.agent, rule = limites_tecnicos_rule)

def balanco_rule(model, agente1, agente2):
    if agente1 >= agente2:
        return Constraint.Skip
    elif model.agent_sign[agente1] == model.agent_sign[agente2]:
        return Constraint.Skip
    else:
        return model.p[agente1, agente2] == model.p[agente2, agente1]
model.balanco = Constraint(model.agent, model.agent, rule = balanco_rule)

def no_arbitrage_rule(model, agente1, agente2):
    if model.agent_sign[agente1] == model.agent_sign[agente2]:
        return model.p[agente1, agente2] == 0
    else:
        return Constraint.Skip
model.no_arbitrage = Constraint(model.agent, model.agent, rule = no_arbitrage_rule)

In [15]:
#Product Differentiation
c = [0, -0.5, -2, -5]
gamma_13 = [10]
gamma_14 = [2]

#Função Objetivo:
def objective_rule(model):
    custo1= sum(model.P[i] * model.c_ener[i] * model.agent_sign[i] for i in model.agent)
    prod_diff = c[3] * gamma_13[0] * model.p['Cons 1', 'Prod A'] + c[3] * gamma_14[0] * model.p['Cons 1', 'Prod B']
    return custo1 + prod_diff
model.objective = Objective(rule = objective_rule, sense = maximize)

In [16]:
solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')
model.dual = Suffix(direction=Suffix.IMPORT)
resultados = solver.solve(model, tee=False)

In [17]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

#Variáveis:
for i in model.agent:
    for j in model.agent:
        print(f' Energia do {i} para {j}: {value(model.p[i,j])}')
    print('\n')

#Print dos CMO:
for (i, j) in model.balanco:
    print(f'Dual para restrição de balanço {i}-{j}: {model.dual[model.balanco[i, j]]}')

#Produção Individual dos Agentes:
for i in model.agent:
    print(f'Produção do {i}: {value(model.P[i])}')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 1950.0 

 Energia do Cons 1 para Cons 1: 0.0
 Energia do Cons 1 para Cons 2: 0.0
 Energia do Cons 1 para Prod A: 0.0
 Energia do Cons 1 para Prod B: 80.0


 Energia do Cons 2 para Cons 1: 0.0
 Energia do Cons 2 para Cons 2: 0.0
 Energia do Cons 2 para Prod A: 50.0
 Energia do Cons 2 para Prod B: 0.0


 Energia do Prod A para Cons 1: 0.0
 Energia do Prod A para Cons 2: 50.0
 Energia do Prod A para Prod A: 0.0
 Energia do Prod A para Prod B: 0.0


 Energia do Prod B para Cons 1: 80.0
 Energia do Prod B para Cons 2: 0.0
 Energia do Prod B para Prod A: 0.0
 Energia do Prod B para Prod B: 0.0


Dual para restrição de balanço Cons 1-Prod A: 12.0
Dual para restrição de balanço Cons 1-Prod B: 30.0
Dual para restrição de balanço Cons 2-Prod A: 12.0
Dual para restrição de balanço Cons 2-Prod B: 30.0
Produção do Cons 1: 80.0
Produção do Cons 2: 50.0
Produção do Prod A: 50.0
Produção do Prod B: 8

In [18]:
#Função Objetivo Recalculada:
FOB = sum(value(model.P[i]) * model.c_ener[i] * model.agent_sign[i] for i in model.agent)
print(f'Social Welfare: {FOB}')

Social Welfare: 2750.0
